In [6]:
# Import necessary modules from PyTorch
import torchvision
from torchvision import transforms

# Data augmentation and normalization
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Define dataset root directory
data_dir = 'dataset_flower102/'

# Load train, validation, and test datasets
train_dataset = torchvision.datasets.Flowers102(root=data_dir, split='train', transform=data_transforms, download=False)
valid_dataset = torchvision.datasets.Flowers102(root=data_dir, split='val', transform=data_transforms, download=True)
test_dataset = torchvision.datasets.Flowers102(root=data_dir, split='test', transform=data_transforms, download=True)

# Create data loaders
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)